In [2]:
import torch
import statsmodels.api as sm
from torch.nn import MSELoss
from sklearn.metrics import roc_auc_score
#torch.nn.functional.sigmoid(torch.tensor(9))

In [1]:
# num_features = 5
# num_observations = 10000
import pandas as pd
pd.Categorical

In [3]:
#binomial_dist = torch.distributions.bernoulli()
num_features = 5
num_observations = 10000
x = torch.rand(size=(num_observations, num_features))
weights_original = torch.randn(size=(1,num_features)).float()
bias_original = torch.randn(size=(1,)).float()
y_linear = x @ weights_original.T + bias_original
y_target = torch.bernoulli(torch.sigmoid(y_linear))
weights_original

tensor([[-0.9991, -1.3117, -0.1253,  1.7374, -0.6081]])

In [4]:
## do logistic regression
# Example dataset
x_numpy = x.detach().numpy()
x_numpy= sm.add_constant(x_numpy)  # Adds a constant (intercept) term
y_numpy = y_target.detach().numpy()

# Fit logistic regression model using GLM with a logit link
glm_model = sm.GLM(y_numpy, x_numpy, family=sm.families.Binomial())
result = glm_model.fit()

# Print summary of the model
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9994
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -6173.8
Date:                Wed, 18 Sep 2024   Deviance:                       12348.
Time:                        20:28:53   Pearson chi2:                 1.00e+04
No. Iterations:                     4   Pseudo R-squ. (CS):            0.09955
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2454      0.085      2.872      0.0

In [5]:
print(result.params, weights_original)
y_pred_prob = result.predict(x_numpy)
# Compute AUC using scikit-learn
auc = roc_auc_score(y_numpy, y_pred_prob)
print(f"AUC: {auc}")

[ 0.24536143 -1.07453786 -1.3518533  -0.12625479  1.68456507 -0.59533207] tensor([[-0.9991, -1.3117, -0.1253,  1.7374, -0.6081]])
AUC: 0.6852583886469651


In [12]:
lr = 0.001
epochs = 1000000
batch_size = 256
loss_f = torch.nn.BCELoss()
sigmoid_f = torch.nn.Sigmoid()
weights_estim_torch = torch.rand(size=(1,num_features), requires_grad=True)
bias_estim_torch = torch.rand(size=(1,), requires_grad=True)

In [13]:
for cur_epoch in range(epochs):
    cur_batch_index = torch.randint(low=0,high=x.shape[0], size=(batch_size,))
    cur_batch_x = x[cur_batch_index,]
    cur_batch_y = y_target[cur_batch_index,]
    cur_batch_pred = torch.sigmoid( cur_batch_x @ weights_estim_torch.T + bias_estim_torch)
    cur_batch_loss = loss_f(cur_batch_pred, cur_batch_y)
    if cur_epoch % 5000 == 0:
        print(f"Loss of current batch is: {cur_batch_loss.item()}")
    cur_batch_loss.backward()
    with torch.no_grad():
        weights_estim_torch -= lr * weights_estim_torch.grad
        bias_estim_torch -= lr* bias_estim_torch.grad
    ## avoid gradient accumaltion
    weights_estim_torch.grad.zero_()
    bias_estim_torch.grad.zero_()

Loss of current batch is: 1.2789621353149414
Loss of current batch is: 0.6697881817817688
Loss of current batch is: 0.6690037846565247
Loss of current batch is: 0.6821731925010681
Loss of current batch is: 0.6506352424621582
Loss of current batch is: 0.6659315824508667
Loss of current batch is: 0.6486377716064453
Loss of current batch is: 0.601770281791687
Loss of current batch is: 0.6414358019828796
Loss of current batch is: 0.6503137946128845
Loss of current batch is: 0.6546835899353027
Loss of current batch is: 0.6184699535369873
Loss of current batch is: 0.6208236813545227
Loss of current batch is: 0.6033412218093872
Loss of current batch is: 0.610804557800293
Loss of current batch is: 0.6160153150558472
Loss of current batch is: 0.6269715428352356
Loss of current batch is: 0.6285388469696045
Loss of current batch is: 0.6071764230728149
Loss of current batch is: 0.5790933966636658
Loss of current batch is: 0.6262401342391968
Loss of current batch is: 0.615852415561676
Loss of curre

In [8]:
#torch.rand(size=(4,2)) + torch.rand(size=(1,))
weights_estim_torch

tensor([[-1.0763, -1.3520, -0.1244,  1.6834, -0.5995]], requires_grad=True)

In [9]:
#weights_estim_torch
#tensor([[ 1.6484, -1.2829, -0.2156,  1.5310, -1.4347]])
#tensor([[-0.9991, -1.3117, -0.1253,  1.7374, -0.6081]])

In [19]:
auc_torch = roc_auc_score(y_target.numpy(), torch.sigmoid( x @ weights_estim_torch.T + bias_estim_torch).detach().numpy())

In [21]:
auc_torch

np.float64(0.6852636733888584)